# Some experements

In [1]:
import requests
import bs4

In [2]:
url = "https://lotv.spawningtool.com"
build_pattern = "/build/"

In [3]:
pages = requests.get(url + build_pattern)
pages_parser = bs4.BeautifulSoup(pages.text, "lxml")

In [4]:
builds_data = pages_parser.find('tbody').find_all('tr')
for e in builds_data[:2]:
    params = e.find_all('td')
    print(params[1].find('a').attrs['href'])
    print(params[1].find('a').text)
    print(params[2].text)
    print(params[3].text)
    print("-"*15)

/build/63519/
DnS' 3gate blink pressure into double forge
PvT
Economic
---------------
/build/63104/
Stats' 2 Oracle archon drop into IAC
PvZ
Economic
---------------


In [5]:
builds_data[0].find('a').attrs['href']

'/build/63519/'

In [6]:
slug = "/build/63026/"
url_page = url + slug
url_page

'https://lotv.spawningtool.com/build/63026/'

In [7]:
page_html = requests.get(url_page)
build_parser = bs4.BeautifulSoup(page_html.text, "lxml")
build = build_parser.find("table").find_all('tr')

In [8]:
build_parser.find("h1").text

'birdy´s - ZvP - 3 Base Roach Ravager Ling Allin (ZvP All-In)'

In [9]:
build[:5]

[<tr><td>  12</td><td>  0:00</td><td>  <nobr><span class="Worker">Drone</span></nobr></td><td>  </td></tr>,
 <tr><td>  13</td><td>  0:13</td><td>  <nobr><span class="Unit">Overlord</span></nobr></td><td>  </td></tr>,
 <tr><td>  13</td><td>  0:19</td><td>  <nobr><span class="Worker">Drone</span></nobr></td><td>  </td></tr>,
 <tr><td>  14</td><td>  0:32</td><td>  <nobr><span class="Worker">Drone x2</span></nobr></td><td>  </td></tr>,
 <tr><td>  16</td><td>  0:50</td><td>  <nobr><span class="Building">Hatchery</span></nobr></td><td>  </td></tr>]

In [10]:
print(build[0].find(class_="Worker") == None)
print(build[0].find(class_="Unit") == None)

False
True


In [11]:
a = build[0]
b = a.find_all('td')
e = b[2]
e.find(class_="Worker")

<span class="Worker">Drone</span>

In [12]:
len(b)

4

In [13]:
c = b[1]
d = c.text[2:]#delete spaces before timing
c, d

(<td>  0:00</td>, '0:00')

In [14]:
f = a.find('span')
f.attrs["class"]

['Worker']

In [15]:
for e in build_parser.find_all('ul')[5].find_all('li')[1:]:
    print(e)

<li>Published on: Dec 20, 2017</li>
<li>Modified on: Dec 29, 2017</li>
<li>Patch: 4.0.0</li>


In [16]:
import re
text = build_parser.find_all('ul')[5].find_all('li')[2].text
re.search(r'Modified', text)

<_sre.SRE_Match object; span=(0, 8), match='Modified'>

In [17]:
text[13:]

'Dec 29, 2017'

In [18]:
text = build_parser.find_all('ul')[5].find_all('li')[1].text
re.search(r'Published', text)

<_sre.SRE_Match object; span=(0, 9), match='Published'>

In [19]:
text[14:]

'Dec 20, 2017'

In [20]:
import pandas as pd
df = pd.DataFrame(columns=["title", "build-type", "published-date", "modified-date", "matchup", "build"])
df = df.astype({'title': 'str', 'build-type': 'str', 'matchup': 'str', 'published-date': 'datetime64[ns]', 'modified-date': 'datetime64[ns]'}, copy=False)

In [21]:
df = df.append({'title': 'title1', 'build-type': 'macro', 'modified-date': pd.to_datetime('Dec 23, 2017')}, ignore_index=True)
df

,title,build-type,published-date,modified-date,matchup,build
0,title1,macro,NaT,2017-12-23,NaN,NaN


In [22]:
pd.to_datetime('Dec 23, 2017')

Timestamp('2017-12-23 00:00:00')

In [23]:
df.at[0, "build"] =  {1: 'askd', 2: '23'}
df

,title,build-type,published-date,modified-date,matchup,build
0,title1,macro,NaT,2017-12-23,NaN,"{1: 'askd', 2: '23'}"


# Functions for parse

## Parse one page

In [24]:
import numpy as np

In [25]:
def parse_date(build_parser):
    date = {}
    for e in build_parser.find_all('li'):
        if (re.search(r'Modified', e.text)):
            date["modified"] = e.text[13:]
        elif (re.search(r'Published', e.text)):
            date["published"] = e.text[14:]
    return date

In [26]:
def parse_limit(row):
    data = row[0].text[2:]
    if (data == ""):
        limit = np.NaN #limit can be empty
    else:
        limit = int(data)
    return limit

In [27]:
def parse_page(slug):
    page_url = url + slug
    build_data = []
    page_html = requests.get(page_url)
    page_html.close()
    build_parser = bs4.BeautifulSoup(page_html.text, "lxml")
    build = build_parser.find("table").find_all('tr')

    title = build_parser.find("h1").text
    date = parse_date(build_parser)
    
    for build_row in build:
        row = build_row.find_all('td')
        if (len(row) == 0):
            continue
            
        row_data = {}
        row_data["limit"] = parse_limit(row)
        row_data["timing"] = row[1].text[2:]
        row_data["action"] = row[-2].text[2:]
        row_data["type"] = row[2].find('span').attrs["class"][0]
        build_data.append(row_data)

    return title, date, build_data

## Parse a few pages

In [28]:
import re

def parse_pages(builds_number=100):
    page_number = 1
    builds = []
    while(builds_number != 0):
        pages = requests.get(url + build_pattern, params={"p": page_number})
        pages.close()
        pages_parser = bs4.BeautifulSoup(pages.text, "lxml")
        page_number += 1
        
        builds_data = pages_parser.find('tbody').find_all('tr')
        for build_data in builds_data:
            params = build_data.find_all('td')
            slug = params[1].find('a').attrs['href']
            
            matchup = params[2].text
            build_type = params[3].text
            
            build = parse_page(slug)
            builds.append((slug, matchup, build_type, build))
            
            builds_number -= 1
            if (builds_number == 0):
                break
                
    return builds

# Functions for represent

## Represent in dataFrame

In [29]:
import pandas as pd

In [30]:
def convert_builds_to_dataframe(builds):
    df = pd.DataFrame(columns=["slug", "title", "matchup", "build-type", "published-date", "modified-date", "build"])
    df = df.astype({'published-date': 'datetime64[ns]', 'modified-date': 'datetime64[ns]'}, copy=False)
    
    index = 0
    for build in builds:
        df = df.append({'slug': build[0], 'title': build[3][0], 'matchup': build[1], 'build-type': build[2], 'published-date': pd.to_datetime(build[3][1]['published'])}, ignore_index=True)
        if ("modified" in build[3][1]):
            df.at[index, "modified-date"] = pd.to_datetime(build[3][1]['modified'])
            
        df.at[index, "build"] = build[3][2]
        index += 1
    
    return df

## Represent in string

In [31]:
WORKER_TYPE = "Worker"
BUILDING_TYPE = "Building"
UNIT_TYPE = "Unit"
UPGRADE_TYPE = "Upgrade"

In [32]:
def convert_builds_to_str(build, show_workers=False, show_buildings=True, show_units=True, show_upgrades=True): #build - series
    result = ""
    result += "link:\t" + url + build["slug"] + "\n"
    result += "title:\t" + build["title"] + "\n"
    result += "matchup:\t" + build["matchup"] + "\n"
    result += "type:\t" + build["build-type"] + "\n"
    result += "published date:\t" + str(build["published-date"].date()) + "\n"
    if (build["modified-date"] is not pd.NaT):
        result += "modified date:\t" + str(build["modified-date"].date()) + "\n"

    for row in build["build"]:
        if ((show_workers and row["type"]==WORKER_TYPE) or
            (show_buildings and row["type"]==BUILDING_TYPE) or
            (show_units and row["type"]==UNIT_TYPE) or
            (show_upgrades and row["type"]==UPGRADE_TYPE)):
            
            if (row["limit"] is not np.NaN):
                result += str(row["limit"])
            result += "\t" + row["timing"] + "\t" + row["action"] + "\n"
            
    return result

## Parse new builds

In [33]:
def add_new_builds(df):
    page_number = 1
    builds = []
    check_duplicate = False
    
    while(not check_duplicate):
        pages = requests.get(url + build_pattern, params={"p": page_number})
        pages.close()
        pages_parser = bs4.BeautifulSoup(pages.text, "lxml")
        page_number += 1
        
        builds_data = pages_parser.find('tbody').find_all('tr')
        for build_data in builds_data:
            params = build_data.find_all('td')
            slug = params[1].find('a').attrs['href']
            matchup = params[2].text
            build_type = params[3].text
            build = parse_page(slug)
            
            title = build[0]
            if ((df["title"] == title).sum() == 0):
                builds.append((slug, matchup, build_type, build)) 
            else:
                check_duplicate = True
                break
                
    return builds

# Tests

## Parse page test

In [34]:
parse_page("/build/62816/")

('ZvT | 12 Pool Ravager (ZvT Cheese)',
 {'modified': 'Dec 12, 2017', 'published': 'Dec 12, 2017'},
 [{'action': 'Spawning Pool',
   'limit': 12,
   'timing': '0:19',
   'type': 'Building'},
  {'action': 'Extractor', 'limit': 14, 'timing': '0:40', 'type': 'Building'},
  {'action': 'Extractor', 'limit': 14, 'timing': '0:45', 'type': 'Building'},
  {'action': 'Overlord', 'limit': 14, 'timing': '0:59', 'type': 'Unit'},
  {'action': 'Roach Warren',
   'limit': 14,
   'timing': '1:08',
   'type': 'Building'},
  {'action': 'Overlord', 'limit': 14, 'timing': '1:30', 'type': 'Unit'},
  {'action': 'Roach x2', 'limit': 14, 'timing': '1:51', 'type': 'Unit'},
  {'action': 'Roach', 'limit': 14, 'timing': '1:55', 'type': 'Unit'},
  {'action': 'Overlord', 'limit': 28, 'timing': '2:38', 'type': 'Unit'}])

## Parse few pages test

In [35]:
builds = parse_pages(15)

In [36]:
builds[0][3][0]

"DnS' 3gate blink pressure into double forge (PvT Economic)"

## Represent in dataFrame test

In [37]:
df = convert_builds_to_dataframe(builds)
df

,slug,title,matchup,build-type,published-date,modified-date,build
0,/build/63519/,DnS' 3gate blink pressure into double forge (P...,PvT,Economic,2017-12-31,NaT,"[{'limit': 12, 'timing': '0:00', 'action': 'Pr..."
1,/build/63104/,Stats' 2 Oracle archon drop into IAC (PvZ Econ...,PvZ,Economic,2017-12-23,NaT,"[{'limit': 12, 'timing': '0:00', 'action': 'Pr..."
2,/build/63026/,birdy´s - ZvP - 3 Base Roach Ravager Ling Alli...,ZvP,All-In,2017-12-20,2017-12-29,"[{'limit': 12, 'timing': '0:00', 'action': 'Dr..."
3,/build/62798/,Railgan ZvT / ZvP - 2 Base Swarmhost Nydus (Zv...,ZvT,Timing Attack,2017-12-11,2017-12-11,"[{'limit': 12, 'timing': '0:01', 'action': 'Dr..."
4,/build/62730/,Innovation's TvP Cyclone Mech (TvP Economic),TvP,Economic,2017-12-08,2017-12-21,"[{'limit': 12, 'timing': '0:00', 'action': 'SC..."
5,/build/63553/,TvZ rush (TvZ Timing Attack),TvZ,Timing Attack,2017-12-31,2017-12-31,"[{'limit': 12, 'timing': '0:01', 'action': 'SC..."
6,/build/63550/,TvT rush (TvT Timing Attack),TvT,Timing Attack,2017-12-31,2017-12-31,"[{'limit': 12, 'timing': '0:01', 'action': 'SC..."
7,/build/63457/,Terran standard build (TvX Economic),TvX,Economic,2017-12-27,2017-12-27,"[{'limit': 12, 'timing': '0:01', 'action': 'SC..."
8,/build/63496/,birdy's 3 base linghydra timing (WorksWithOver...,ZvP,Timing Attack,2017-12-29,2017-12-29,"[{'limit': 13, 'timing': '0:12', 'action': 'Ov..."
9,/build/63460/,Zerg standard build (ZvX Economic),ZvX,Economic,2017-12-27,2017-12-27,"[{'limit': 12, 'timing': '0:01', 'action': 'Dr..."


## Represent in string test

In [38]:
convert_builds_to_str(df.loc[2])

'link:\thttps://lotv.spawningtool.com/build/63026/\ntitle:\tbirdy´s - ZvP - 3 Base Roach Ravager Ling Allin (ZvP All-In)\nmatchup:\tZvP\ntype:\tAll-In\npublished date:\t2017-12-20\nmodified date:\t2017-12-29\n13\t0:13\tOverlord\n16\t0:50\tHatchery\n18\t1:09\tExtractor\n17\t1:15\tSpawning Pool\n19\t1:48\tHatchery\n19\t2:00\tOverlord\n19\t2:05\tZergling x4\n21\t2:18\tQueen\n25\t2:31\tMetabolic Boost\n25\t2:44\tRoach Warren\n25\t3:10\tOverlord\n27\t3:13\tZergling x6\n30\t3:26\tRoach x5\n42\t3:34\tRoach x2\n44\t3:43\tOverlord\n44\t3:45\tZergling x4\n46\t3:54\tZergling x6\n53\t4:13\tOverlord\n53\t4:21\tRavager\n59\t4:52\tRavager\n52\t5:31\tQueen\n52\t5:33\tRoach\n54\t5:45\tRoach\n53\t6:14\tRoach\n'

## Parse new builds test

In [39]:
df = df.drop(0)
df.head()

,slug,title,matchup,build-type,published-date,modified-date,build
1,/build/63104/,Stats' 2 Oracle archon drop into IAC (PvZ Econ...,PvZ,Economic,2017-12-23,NaT,"[{'limit': 12, 'timing': '0:00', 'action': 'Pr..."
2,/build/63026/,birdy´s - ZvP - 3 Base Roach Ravager Ling Alli...,ZvP,All-In,2017-12-20,2017-12-29,"[{'limit': 12, 'timing': '0:00', 'action': 'Dr..."
3,/build/62798/,Railgan ZvT / ZvP - 2 Base Swarmhost Nydus (Zv...,ZvT,Timing Attack,2017-12-11,2017-12-11,"[{'limit': 12, 'timing': '0:01', 'action': 'Dr..."
4,/build/62730/,Innovation's TvP Cyclone Mech (TvP Economic),TvP,Economic,2017-12-08,2017-12-21,"[{'limit': 12, 'timing': '0:00', 'action': 'SC..."
5,/build/63553/,TvZ rush (TvZ Timing Attack),TvZ,Timing Attack,2017-12-31,2017-12-31,"[{'limit': 12, 'timing': '0:01', 'action': 'SC..."


In [40]:
new_builds = add_new_builds(df)
new_df = convert_builds_to_dataframe(new_builds)
df = pd.concat([new_df, df])

In [41]:
df.head()

,slug,title,matchup,build-type,published-date,modified-date,build
0,/build/63519/,DnS' 3gate blink pressure into double forge (P...,PvT,Economic,2017-12-31,NaT,"[{'limit': 12, 'timing': '0:00', 'action': 'Pr..."
1,/build/63104/,Stats' 2 Oracle archon drop into IAC (PvZ Econ...,PvZ,Economic,2017-12-23,NaT,"[{'limit': 12, 'timing': '0:00', 'action': 'Pr..."
2,/build/63026/,birdy´s - ZvP - 3 Base Roach Ravager Ling Alli...,ZvP,All-In,2017-12-20,2017-12-29,"[{'limit': 12, 'timing': '0:00', 'action': 'Dr..."
3,/build/62798/,Railgan ZvT / ZvP - 2 Base Swarmhost Nydus (Zv...,ZvT,Timing Attack,2017-12-11,2017-12-11,"[{'limit': 12, 'timing': '0:01', 'action': 'Dr..."
4,/build/62730/,Innovation's TvP Cyclone Mech (TvP Economic),TvP,Economic,2017-12-08,2017-12-21,"[{'limit': 12, 'timing': '0:00', 'action': 'SC..."
